# Statistical Rethinking: Excercise 2M2

Like in the previous notebook for 2M1, we have the following globe tossing results:

(1) W, W, W <br>
(2) W, W, W, L <br>
(3) L, W, W, L, W, W, W <br>

<br>
We want to estimate the probability of Water $p$ as parameter of the model. The parameter is at least 0 and at most 1.
<br>
<br>
The key point for this exercise is that we employ a different prior:
$$    \text{prior}(p) =
    \begin{cases}
      0, & \text{if}\ p < 0.5 \\
      1, & \text{otherwise}
    \end{cases}$$

In [1]:
require "rubythinking"
grid_size = 50
step_size = 1.0 / grid_size.to_f
grid = 0.step(by: step_size, to: 1).to_a

[0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2, 0.22, 0.24, 0.26, 0.28, 0.3, 0.32, 0.34, 0.36, 0.38, 0.4, 0.42, 0.44, 0.46, 0.48, 0.5, 0.52, 0.54, 0.56, 0.58, 0.6, 0.62, 0.64, 0.66, 0.68, 0.7000000000000001, 0.72, 0.74, 0.76, 0.78, 0.8, 0.8200000000000001, 0.84, 0.86, 0.88, 0.9, 0.92, 0.9400000000000001, 0.96, 0.98, 1.0]

In [3]:
prior = grid.map do |x| 
  y =
    if x < 0.5
      0
    else
      1
    end
  
  {x: x, y: y}
end

Vega.lite
  .data(prior)
  .mark(type: "area")
  .encoding(
    x: {field: "x", type: "quantitative"},
    y: {field: "y", type: "quantitative", scale: {"domain": [0,2]}}
  )

#<Vega::LiteChart:0x00007f9f568c61d8 @schema="https://vega.github.io/schema/vega-lite/v4.json", @spec={"$schema"=>"https://vega.github.io/schema/vega-lite/v4.json", "width"=>"container", "height"=>"container", "data"=>{:values=>[{:x=>0.0, :y=>0}, {:x=>0.02, :y=>0}, {:x=>0.04, :y=>0}, {:x=>0.06, :y=>0}, {:x=>0.08, :y=>0}, {:x=>0.1, :y=>0}, {:x=>0.12, :y=>0}, {:x=>0.14, :y=>0}, {:x=>0.16, :y=>0}, {:x=>0.18, :y=>0}, {:x=>0.2, :y=>0}, {:x=>0.22, :y=>0}, {:x=>0.24, :y=>0}, {:x=>0.26, :y=>0}, {:x=>0.28, :y=>0}, {:x=>0.3, :y=>0}, {:x=>0.32, :y=>0}, {:x=>0.34, :y=>0}, {:x=>0.36, :y=>0}, {:x=>0.38, :y=>0}, {:x=>0.4, :y=>0}, {:x=>0.42, :y=>0}, {:x=>0.44, :y=>0}, {:x=>0.46, :y=>0}, {:x=>0.48, :y=>0}, {:x=>0.5, :y=>1}, {:x=>0.52, :y=>1}, {:x=>0.54, :y=>1}, {:x=>0.56, :y=>1}, {:x=>0.58, :y=>1}, {:x=>0.6, :y=>1}, {:x=>0.62, :y=>1}, {:x=>0.64, :y=>1}, {:x=>0.66, :y=>1}, {:x=>0.68, :y=>1}, {:x=>0.7000000000000001, :y=>1}, {:x=>0.72, :y=>1}, {:x=>0.74, :y=>1}, {:x=>0.76, :y=>1}, {:x=>0.78, :y=>1}, {:x=>0.8, :y=>1}, {:x=>0.8200000000000001, :y=>1}, {:x=>0.84, :y=>1}, {:x=>0.86, :y=>1}, {:x=>0.88, :y=>1}, {:x=>0.9, :y=>1}, {:x=>0.92, :y=>1}, {:x=>0.9400000000000001, :y=>1}, {:x=>0.96, :y=>1}, {:x=>0.98, :y=>1}, {:x=>1.0, :y=>1}]}, "mark"=>{:type=>"area"}, "encoding"=>{:x=>{:field=>"x", :type=>"quantitative"}, :y=>{:field=>"y", :type=>"quantitative", :scale=>{:domain=>[0, 2]}}}}>

In [5]:
factorial = ->(n) do
  return 1 if n < 1
  n.to_i.downto(1).inject(:*)
end


likelihood = ->(w, l, p) do
  (factorial[w+l].to_f / (factorial[w] * factorial[l])).to_f * (p**w) * ((1-p)**l)
end

#<Proc:0x00007f9f50846600@<main>:6 (lambda)>

Now, let's compute the grid aprroximation of the posterior for each of the cases. The difference is only the data input we give in terms of "count of Water" versus "count of Land" of our tossing result given in the exercise.

In [22]:
# For case (1)
w = 3
l = 0

print_posterior = ->(w, l) do
  u_posterior = grid.map do |x| 
    prior_x = prior.detect { |p| p[:x] == x }[:x]
    y = prior_x * likelihood[w, l, x]
    {x: x, y: y}
  end

  posterior = u_posterior.map do |pair| 
    y = pair[:y].to_f / u_posterior.map { |pair| pair[:y] }.sum.to_f
   {x: pair[:x], y: y}
  end

  Vega.lite
    .data(posterior)
    .mark(type: "area")
    .encoding(
      x: {field: "x", type: "quantitative"},
      y: {field: "y", type: "quantitative", scale: {"domain": [0,0.1]}}
    )
end

print_posterior[3, 0]

#<Vega::LiteChart:0x00007f9f5113e348 @schema="https://vega.github.io/schema/vega-lite/v4.json", @spec={"$schema"=>"https://vega.github.io/schema/vega-lite/v4.json", "width"=>"container", "height"=>"container", "data"=>{:values=>[{:x=>0.0, :y=>0.0}, {:x=>0.02, :y=>1.5228426782447382e-08}, {:x=>0.04, :y=>2.436548285191581e-07}, {:x=>0.06, :y=>1.2335025693782375e-06}, {:x=>0.08, :y=>3.89847725630653e-06}, {:x=>0.1, :y=>9.517766739029615e-06}, {:x=>0.12, :y=>1.97360411100518e-05}, {:x=>0.14, :y=>3.6563452704656174e-05}, {:x=>0.16, :y=>6.237563610090448e-05}, {:x=>0.18, :y=>9.991370811963724e-05}, {:x=>0.2, :y=>0.00015228426782447384}, {:x=>0.22, :y=>0.00022295939652181208}, {:x=>0.24, :y=>0.0003157766577608288}, {:x=>0.26, :y=>0.0004349390973334797}, {:x=>0.28, :y=>0.0005850152432744988}, {:x=>0.3, :y=>0.0007709391058613984}, {:x=>0.32, :y=>0.0009980101776144717}, {:x=>0.34, :y=>0.0012718934332967878}, {:x=>0.36, :y=>0.001598619329914196}, {:x=>0.38, :y=>0.0019845838067153255}, {:x=>0.4, :y=>0.0024365482851915815}, {:x=>0.42, :y=>0.002961639669077148}, {:x=>0.44, :y=>0.0035673503443489933}, {:x=>0.46, :y=>0.004261538179226858}, {:x=>0.48, :y=>0.005052426524173261}, {:x=>0.5, :y=>0.005948604211893508}, {:x=>0.52, :y=>0.006959025557335675}, {:x=>0.54, :y=>0.00809301035769062}, {:x=>0.56, :y=>0.00936024389239198}, {:x=>0.58, :y=>0.010770776923116163}, {:x=>0.6, :y=>0.012335025693782375}, {:x=>0.62, :y=>0.014063771930552586}, {:x=>0.64, :y=>0.015968162841831546}, {:x=>0.66, :y=>0.01805971111826678}, {:x=>0.68, :y=>0.020350294932748605}, {:x=>0.7000000000000001, :y=>0.02285215794041011}, {:x=>0.72, :y=>0.025577909278627135}, {:x=>0.74, :y=>0.02854052356701836}, {:x=>0.76, :y=>0.03175334090744521}, {:x=>0.78, :y=>0.03523006688401185}, {:x=>0.8, :y=>0.038984772563065304}, {:x=>0.8200000000000001, :y=>0.0430318944931953}, {:x=>0.84, :y=>0.04738623470523437}, {:x=>0.86, :y=>0.05206296071225788}, {:x=>0.88, :y=>0.05707760550958389}, {:x=>0.9, :y=>0.0624460675747733}, {:x=>0.92, :y=>0.06818461086762972}, {:x=>0.9400000000000001, :y=>0.07430986483019963}, {:x=>0.96, :y=>0.08083882438677217}, {:x=>0.98, :y=>0.08778884994387942}, {:x=>1.0, :y=>0.09517766739029612}]}, "mark"=>{:type=>"area"}, "encoding"=>{:x=>{:field=>"x", :type=>"quantitative"}, :y=>{:field=>"y", :type=>"quantitative", :scale=>{:domain=>[0, 0.1]}}}}>

In [23]:
# For case (2)
w = 3
l = 1
print_posterior[3, 1]

#<Vega::LiteChart:0x00007f9f568964b0 @schema="https://vega.github.io/schema/vega-lite/v4.json", @spec={"$schema"=>"https://vega.github.io/schema/vega-lite/v4.json", "width"=>"container", "height"=>"container", "data"=>{:values=>[{:x=>0.0, :y=>0.0}, {:x=>0.02, :y=>9.417415154975339e-08}, {:x=>0.04, :y=>1.4760356814328695e-06}, {:x=>0.06, :y=>7.316754998977776e-06}, {:x=>0.08, :y=>2.2632547115304e-05}, {:x=>0.1, :y=>5.405404106809825e-05}, {:x=>0.12, :y=>0.00010959564934639052}, {:x=>0.14, :y=>0.00019842493731533044}, {:x=>0.16, :y=>0.00033063199264096277}, {:x=>0.18, :y=>0.0005169987947150043}, {:x=>0.2, :y=>0.0007687685840796197}, {:x=>0.22, :y=>0.0010974152318521965}, {:x=>0.24, :y=>0.0015144126091501235}, {:x=>0.26, :y=>0.002031003956515566}, {:x=>0.28, :y=>0.00265797125334024}, {:x=>0.3, :y=>0.0034054045872901886}, {:x=>0.32, :y=>0.004282471523730565}, {:x=>0.34, :y=>0.005297186475150397}, {:x=>0.36, :y=>0.00645618007058737}, {:x=>0.38, :y=>0.007764468525052607}, {:x=>0.4, :y=>0.009225223008955436}, {:x=>0.42, :y=>0.010839539017528162}, {:x=>0.44, :y=>0.012606205740250873}, {:x=>0.46, :y=>0.014521475430276175}, {:x=>0.48, :y=>0.016578832773853986}, {:x=>0.5, :y=>0.018768764259756335}, {:x=>0.52, :y=>0.02107852754870209}, {:x=>0.54, :y=>0.02349192084278178}, {:x=>0.56, :y=>0.02598905225488235}, {:x=>0.58, :y=>0.0285461091781119}, {:x=>0.6, :y=>0.03113512765522458}, {:x=>0.62, :y=>0.03372376174804523}, {:x=>0.64, :y=>0.0362750529068942}, {:x=>0.66, :y=>0.03874719934001217}, {:x=>0.68, :y=>0.041093325382984894}, {:x=>0.7000000000000001, :y=>0.04326125086816796}, {:x=>0.72, :y=>0.045193260494111584}, {:x=>0.74, :y=>0.046825873194985414}, {:x=>0.76, :y=>0.04808961151000324}, {:x=>0.78, :y=>0.04890877095284782}, {:x=>0.8, :y=>0.04920118938109565}, {:x=>0.8200000000000001, :y=>0.04887801636564171}, {:x=>0.84, :y=>0.0478434825601243}, {:x=>0.86, :y=>0.04599466907034977}, {:x=>0.88, :y=>0.04322127682371728}, {:x=>0.9, :y=>0.03940539593864362}, {:x=>0.92, :y=>0.03442127509398795}, {:x=>0.9400000000000001, :y=>0.02813509089847664}, {:x=>0.96, :y=>0.020404717260128002}, {:x=>0.98, :y=>0.011079494755676943}, {:x=>1.0, :y=>0.0}]}, "mark"=>{:type=>"area"}, "encoding"=>{:x=>{:field=>"x", :type=>"quantitative"}, :y=>{:field=>"y", :type=>"quantitative", :scale=>{:domain=>[0, 0.1]}}}}>

In [24]:
# For case (3)
w = 5
l = 2
print_posterior[5, 2]

#<Vega::LiteChart:0x00007f9f5584f7f0 @schema="https://vega.github.io/schema/vega-lite/v4.json", @spec={"$schema"=>"https://vega.github.io/schema/vega-lite/v4.json", "width"=>"container", "height"=>"container", "data"=>{:values=>[{:x=>0.0, :y=>0.0}, {:x=>0.02, :y=>3.0978724813463113e-10}, {:x=>0.04, :y=>1.9025401274860552e-08}, {:x=>0.06, :y=>2.0777563627377455e-07}, {:x=>0.08, :y=>1.1182708082668035e-06}, {:x=>0.1, :y=>4.0824082249103786e-06}, {:x=>0.12, :y=>1.165424736843176e-05}, {:x=>0.14, :y=>2.806703446824573e-05}, {:x=>0.16, :y=>5.966365839796267e-05}, {:x=>0.18, :y=>0.00011526412208966084}, {:x=>0.2, :y=>0.0002064388159164557}, {:x=>0.22, :y=>0.0003476615817525607}, {:x=>0.24, :y=>0.0005563217586781971}, {:x=>0.26, :y=>0.0008525796035548661}, {:x=>0.28, :y=>0.001259054681954664}, {:x=>0.3, :y=>0.0018003420271854754}, {:x=>0.32, :y=>0.002502356067412058}, {:x=>0.34, :y=>0.00339150752313115}, {:x=>0.36, :y=>0.004493723679516962}, {:x=>0.38, :y=>0.0058333276404115035}, {:x=>0.4, :y=>0.007431797372992403}, {:x=>0.42, :y=>0.009306430554409032}, {:x=>0.44, :y=>0.01146894643393589}, {:x=>0.46, :y=>0.013924061126450376}, {:x=>0.48, :y=>0.016668077955300224}, {:x=>0.5, :y=>0.019687539664884148}, {:x=>0.52, :y=>0.022957994525527093}, {:x=>0.54, :y=>0.026442933555490128}, {:x=>0.56, :y=>0.030092961287212706}, {:x=>0.58, :y=>0.03384526770714348}, {:x=>0.6, :y=>0.037623474200774026}, {:x=>0.62, :y=>0.04133793153674749}, {:x=>0.64, :y=>0.044886553126173394}, {:x=>0.66, :y=>0.04815627199553666}, {:x=>0.68, :y=>0.05102521511384827}, {:x=>0.7000000000000001, :y=>0.053365693916942326}, {:x=>0.72, :y=>0.05504811507408278}, {:x=>0.74, :y=>0.055945920744301374}, {:x=>0.76, :y=>0.05594167277214611}, {:x=>0.78, :y=>0.054934400474778086}, {:x=>0.8, :y=>0.05284833687461262}, {:x=>0.8200000000000001, :y=>0.04964317343395854}, {:x=>0.84, :y=>0.045325968550367905}, {:x=>0.86, :y=>0.03996485027366636}, {:x=>0.88, :y=>0.033704658907893224}, {:x=>0.9, :y=>0.026784680363636973}, {:x=>0.92, :y=>0.01955862632851189}, {:x=>0.9400000000000001, :y=>0.012517022525778358}, {:x=>0.96, :y=>0.006312171533368142}, {:x=>0.98, :y=>0.001785861837833773}, {:x=>1.0, :y=>0.0}]}, "mark"=>{:type=>"area"}, "encoding"=>{:x=>{:field=>"x", :type=>"quantitative"}, :y=>{:field=>"y", :type=>"quantitative", :scale=>{:domain=>[0, 0.1]}}}}>